In [1]:
import psycopg2
from faker import Faker
import random
import csv


fake = Faker()

In [2]:
def get_connection():
    return psycopg2.connect(
        host="localhost",
        database="5310 project",
        user="postgres",
        password="123"
    )


In [3]:
positive_feedback = [
    "The package arrived well before the estimated delivery date. Excellent service!",
    "I was impressed with the secure packaging and prompt delivery. Everything arrived in perfect condition.",
    "Great experience! The courier was friendly, and the tracking updates were accurate.",
    "Fast and efficient delivery. The package was handled with care.",
    "The delivery process was seamless, and the package arrived without any issues."
]

neutral_feedback = [
    "The delivery was on time, but the packaging could have been better protected.",
    "The delivery was okay, but I wish there were more updates on the tracking information.",
    "The package arrived slightly late, but the product was in good condition.",
    "It was an average experience; nothing stood out, but no issues either.",
    "The delivery was neither too fast nor too slow; overall, a satisfactory experience."
]

negative_feedback = [
    "The package arrived late and was slightly damaged. Not happy with the service.",
    "The courier was rude and left the package at the wrong address.",
    "The delivery was delayed without any explanation or updates.",
    "The package arrived damaged due to poor packaging. Very disappointed.",
    "The tracking information was inaccurate, and the package arrived much later than expected."
]


def generate_feedback(rating):
    if rating >= 4:
        return random.choice(positive_feedback)
    elif rating == 3:
        return random.choice(neutral_feedback)
    else:
        return random.choice(negative_feedback)

def generate_shipping_satisfaction(n=5000):
    conn = get_connection()
    cursor = conn.cursor()

    # Fetch all unique shipping_ids from the shippings table
    cursor.execute("SELECT DISTINCT shipping_id FROM shippings")
    shipping_ids = [shipping[0] for shipping in cursor.fetchall()]  # Flatten the list of tuples

    # Ensure that we do not exceed the number of unique shipping_ids
    if len(shipping_ids) < n:
        print(f"Error: Not enough unique shipping_ids. Need at least {n} unique shipping_ids.")
        cursor.close()
        conn.close()
        return

    # Shuffle the list of shipping_ids to ensure randomness
    random.shuffle(shipping_ids)

    satisfaction_data = []

    for i in range(n):
        shipping_id = shipping_ids[i]
        rating = random.randint(1, 5)  # Rating between 1 and 5
        feedback = generate_feedback(rating)
        satisfaction_data.append((shipping_id, rating, feedback))


    cursor.execute(
      "INSERT INTO shipping_satisfaction (shipping_id, rating, feedback) VALUES (%s, %s, %s)",
       (shipping_id, rating, feedback)
        )

    conn.commit()
    cursor.close()
    conn.close()

    # Write the generated ratings data to a CSV file
    with open('C:/Users/Yihua/Desktop/5310 project/shipping_satisfaction.csv', 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(['shipping_id', 'rating', 'feedback'])
        csv_writer.writerows(satisfaction_data)

    print(f"Generated {len(satisfaction_data)} rows of satisfaction data.")
if __name__ == "__main__":
    generate_shipping_satisfaction()

Generated 5000 rows of satisfaction data.
